In [ ]:
from sympy import *
init_printing()
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

Ungleichungen

In [ ]:
x = Symbol('x')
y = Symbol('y')
a = Symbol('a')

In [ ]:
f = x**2 + x + 1
g = 2*x**2
f, g

In [ ]:
solveset(f>g, domain=Reals)

In [ ]:
print(_)

In [ ]:
solveset(sin(x) > cos(x), domain=Reals)

Die korrekte Lösungsmenge ist allerdings periodisch.

In [ ]:
solve(sin(x) > cos(x), x)

Die `LambertW`-Funktion

In [ ]:
f = x*exp(x)
glg = Eq(f, y)
glg

In [ ]:
solveset(glg, x)

In [ ]:
solve(glg, x)

Wir schauen uns die Funktion $xe^x$ genauer an

In [ ]:
xn = np.linspace(-5, 1.5)
fn = lambdify(x, f, 'numpy')

In [ ]:
plt.plot(xn, fn(xn));

$f$ ist offenbar nicht injektiv.  Wo ist die Minimalstelle?

In [ ]:
df = f.diff(x)
df

In [ ]:
solve(df)

In [ ]:
x0 = solve(df)[0]

In [ ]:
#y0 = fn(x0)
# AttributeError

In [ ]:
type(x0)

Das hätte man in der Renaissance genauso gesehen.

Eigentlich hätten wir $y_0$ aber auch lieber symbolisch.

In [ ]:
y0 = f.subs(x,x0)
x0, y0

In [ ]:
xn = np.linspace(float(y0)+.000001, 7, 300)

Jetzt zeichnen wir den Hauptzweig der `LambertW`-Funktion.  

Es gibt drei Bibliotheken für Mathefunktionen:

* `numpy`:  elemetare Funktionen als ufunc mit doppelter Genauigkeit
* `scipy.special`:  spezielle Funktionen als ufunc mit doppelter Genauigkeit
* `mpmath`:  elementare und spezielle Funktionen mit beliebiger Genauigkeit, aber nicht als ufunc

`Lambertw` ist in `mpmath` und in `scipy.special` (als `lambertw`)

Wir nutzen `lambdify`, obwohl das in diesem einfachen Fall nicht sinnvoll ist.

erste Möglichkeit: List Comprehension

In [ ]:
fn1 = lambdify(y, LambertW(y), 'mpmath')

In [ ]:
fn1(2)

In [ ]:
yn1 = [fn1(xx) for xx in xn]

In [ ]:
plt.figure()
plt.plot(xn, yn1);

zweite Möglichkeit: `scipy.special` nutzen

In [ ]:
from scipy import special
fn2 = lambdify(y, LambertW(y), ['numpy', {'LambertW': special.lambertw}])

In [ ]:
fn2(2)

In [ ]:
yn2 = fn2(xn)

In [ ]:
plt.figure()
plt.plot(xn, yn2);

Jetzt der andere Ast.

In [ ]:
xn_minus = np.linspace(float(y0)+.000001, 0)
fn_minus = lambdify(y, LambertW(y, -1), 'mpmath')
yn_minus = [fn_minus(x) for x in xn_minus]

In [ ]:
plt.figure()
plt.plot(xn, yn2)
plt.plot(xn_minus, yn_minus);

In [ ]:
glg = Eq(2-x, a*log(x))
glg

In [ ]:
f = solve(glg, x)
f

In [ ]:
N(f[0].subs(a, 20))

Wie verhält sich diese Funktion bei Annäherung an $\infty$?

Zu diesem Zweck wird jetzt die Methode `series` vorgestellt

In [ ]:
g = cos(x)
g.series(x, 0)

In [ ]:
g.series(x, 0, 14)

Zur Weiterverarbeitung muss häufig der Ordnungsterm entfernt werden:

In [ ]:
g.series(x, 0, 14).removeO()

In [ ]:
g.series(x, pi/2)

In [ ]:
log(x).series(x, 1)

Hier handelt es sich um Taylorpolynome.

In [ ]:
h = log(x + 1/x)
h.series(x, 0)

Das bedeutet:

Es gibt $C>0$, so dass 

$$ \left| \log\left( x + \frac1x \right) + \log x - x^2 + \frac{x^4}2 \right| \le C x^6
$$
für $ x \searrow 0 $.

In [ ]:
h.series(x, 0, dir='-')

Fragen Sie Dr. Köhne.

Mit derselben Interpretation kann man auch Reihenentwicklungen für $x\to\infty$ bekommen.

In [ ]:
g = (1+x**3+x**2)/(1-x**2)
g

In [ ]:
g.series(x, oo)

In [ ]:
h = log(x + 1/x + 1/x**2)
h.series(x, oo)

In [ ]:
f

In [ ]:
# f[0].series(a, oo)
# RecursionError

Die Reihenentwicklung von `LambertW` in $0$ ist nicht korrekt implementiert.  Das können wir selber machen, indem wir die Taylorkoeffizienten berechnen.  

In [ ]:
b = {}

In [ ]:
b[0] = LambertW(0)
b[0]

In [ ]:
dL = LambertW(y).diff(y)
dL  

In [ ]:
# limit(dL, y, 0)  
# RecursionError

Aber $W(y)e^{W(y)}=y$

In [ ]:
tmp = Symbol('tmp')
dL = dL.subs(LambertW(y), tmp).subs(y, tmp*exp(tmp)).subs(tmp, LambertW(y))
dL

In [ ]:
b[1] = dL.subs(y, 0)
b[1]

In [ ]:
ddL = dL.diff(y)
ddL

In [ ]:
ddL = ddL.subs(LambertW(y), tmp).subs(y, tmp*exp(tmp)).subs(tmp, LambertW(y))
ddL

In [ ]:
b[2] = ddL.subs(y, 0)
b[2]

In [ ]:
Ws = b[0] + b[1]*y + b[2]/2 * y**2 + O(y**3)
Ws

In [ ]:
C = Symbol('C')
def Ws(y):
    return b[0] + b[1]*y + b[2]/2*y**2 + C*y**3 # statt O()

In [ ]:
(Ws(y)**2).expand()

In [ ]:
f[0]

In [ ]:
tmp = f[0].replace(LambertW, Ws)
tmp

In [ ]:
tmp.expand().series(a, oo)

Alle $C$-Term fallen nach $O()$, also

In [ ]:
fs = tmp.expand().series(a, oo, 2)
fs

In [ ]:
fn = lambdify(a, f[0], ['numpy', {'LambertW': special.lambertw }])
fn(.02)

In [ ]:
plt.figure()
xn = np.linspace(3, 30)
yn = fn(xn).real
plt.plot(xn, yn, label='f')
plt.plot(xn, 1+1/xn, label='approx')
plt.legend();